### 3.0 preload raw byte string content from pickle
+ raw contains 45 meeting minutes from San Jose city concoil.

In [1]:
import pickle

def load_obj(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)

In [4]:
raw      = load_obj('council_minutes_dict_new.pkl')
articles = []
for (key,article) in raw.items():
    #article = byte_article.decode('ISO8859-1')
    articles.append(article)

### 3.1 search and separate content as per formal text pattern
+ identify and separate individual main topic and the content between.
+ identify sub topic within the content block.

+ uses regex as main usage.

In [5]:
# import regex and identify patterns for each content of interests.
import regex as re

# room for improvement. host defined pattern to backend/user defined pattern
# may subject to change between different pdf contents
patterns = {'maintitle':re.compile(r'\n{1}\s?(\d{0,2}[.]\s*.*?)\n'),
           'subtitle':re.compile(r'\n{1}\s?(\d?[.]\d?\s*.*)'),
           'dollar':re.compile('([$]\d+([,.]\d+)*)'),
           'item':re.compile(r'\s(\d+-\d+\s+.*)\n'),
           'section':re.compile(r'\s((Sections?)\s\d+[.]\d+[.]\d+)')}

+ identifing recoganizable key words for main title recognition

In [6]:
# 

mainTopicLS =['CEREMONIAL',
              'CEREMONIALS',
              'ITEMS',
              'CONSENT',
              'CALENDAR',
              'STRATEGIC',
              'SUPPORT',
              'COMMUNITY',
             'ECONOMIC',
             'DEVELOPMENT',
             'NEIGHBORHOOD',
             'SERVICES',
             'TRANSPORTATION',
             'AVIATION',
             'ENVIRONMENTAL',
             'UTILITY',
             'PUBLIC',
             'SAFETY',
             'REDEVELOPMENT',
             'SUCCESSOR',
             'AGENCY',
             'LAND',
             'USE']
# The line below removes duplicates
mainTopicLS = list(dict.fromkeys(mainTopicLS)) 

+ main method to turn a string into a dataframe with main and subtitle marked.

In [7]:
import pandas as pd
import math

def qcRework (BLOCKS,pickLS = [],articleID = 1):
    Blocks = [[b,0,0,articleID] for b in BLOCKS]
    Blocks_result = []
    i = 0
    j = 0

    for b in Blocks:
        word_ls = b[0].split(' ')
        word_ls = [w for w in word_ls if w!=' ']
        word_ls = [w for w in word_ls if w!='']
        word_ls = [w for w in word_ls if w!='\n']
        try:
            main = float(word_ls[0])
            sub = round((main-i)*100)
            # print(word_ls[0],sub)
            if sub%10 == 0 and j != 9:
                sub = round(sub/10)
            else:
                pass
            if math.floor(main)==i and sub == j+1:
                j = sub
                #print(word_ls[0],j)
        except:
            pass
        
        if len(word_ls)<2:
            b[1] = j
            b[2] = i
            Blocks_result.append(b)
            continue
        
        if word_ls[-1] == word_ls[-1].upper() and word_ls[1] in pickLS:
            i += 1
            j = 0
            b[2] = i
            Blocks_result.append(b)
            
        else:
            b[1] = j
            b[2] = i
            Blocks_result.append(b)
            
    df_blocks = pd.DataFrame(Blocks_result,columns = ['content','subID','mainID','artID'])
    return df_blocks

+ process all articles in the string list.

In [8]:
articles_in_blocks = []
i = 1

for a in articles:
    #print(i)
    BLOCKS = re.split(patterns['maintitle'],a)
    articles_in_blocks.append(qcRework(BLOCKS,mainTopicLS,i))
    i+=1

+ visual validation

In [9]:
df_test = articles_in_blocks[1]

df_test.iloc[35:50]

,content,subID,mainID,artID
35,2.11 20-067 Retroactive Approval of Gingerbrea...,11,2,2
36,by Council District 10 as a City Council Sp...,11,2,2
37,2.11 20-067 Retroactive Approval of Gingerbrea...,11,2,2
38,\n\n (b) Approve and accept donations from va...,11,2,2
39,2.12 20-074 Approval of Del Roble Beautificati...,12,2,2
40,City Council Sponsored Special Event to Expen...,12,2,2
41,2.13 20-081 Council Agenda Order Change and Ad...,13,2,2
42,\n Rules of Conduct Resolution. \n \n\n Recom...,13,2,2
43,2.14 20-063 Appointment to the Board of Admini...,14,2,2
44,\n\n Recommendation: Appoint one (1) employee...,14,2,2


### 3.2 pack up all the subtitle content into one string.

+ function to identify the all the subtitle content pack into one

In [10]:
def df_pack(df_messy):
    df_compact = pd.DataFrame(columns = df_messy.columns)
    lastrow = None

    for id,row in df_messy.iterrows():
        if lastrow is None:
            lastrow = row
            continue
        if (row['subID'] == lastrow['subID']) and row['mainID'] == lastrow['mainID'] and row['artID'] == lastrow['artID']:
            lastrow['content'] += row['content']
        else:
            df_compact = df_compact.append(lastrow,ignore_index = True)
            lastrow = row
                  
    df_compact = df_compact.append(lastrow,ignore_index = True)
    
    return df_compact

+ process all the articles block list into dataframes.

In [11]:
df_articles = []

for a in articles_in_blocks:
    df_articles.append(df_pack(a))

### 3.3 add logic columns for specific contents

+ findPattern: find all the patterns in a string and return a logic value
+ addColumn: add a logic column for a specific pattern recognition by using findPattern function

In [12]:
def findPattern(s,pattern):
    sub = re.findall(pattern,s)
    #rint(sub)
    return sub !=[]

def addColumn(df,pattern,colname = None):
    if colname is None:
        colname = 'col' + str(len(df.columns))
    
    df[colname] = df['content'].apply(findPattern,args=[pattern])
    return df      


+ add hasDollar, hasItem and hasSection for all the artical dataframe

In [13]:
for df in df_articles:
    df = addColumn(df,patterns['dollar'],'hasDollar')
    df = addColumn(df,patterns['item'],'hasItem')
    df = addColumn(df,patterns['section'],'hasSection')


+ visual validation.

### 3.4 join all dataset in one dataframe

In [14]:
df_minutes =pd.DataFrame(columns= df_articles[0].columns)
df_minutes = df_minutes.append(df_articles,ignore_index = True)
df_minutes

,content,subID,mainID,artID,hasDollar,hasItem,hasSection
0,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,0,0,1,False,True,False
1,1. CEREMONIAL ITEMSNone provided.\n\nCity of S...,0,1,1,False,False,False
2,2. CONSENT CALENDAR\nUpon motion by Councilmem...,0,2,1,False,False,False
3,2.1 Approval of City Council Minutes.None prov...,1,2,1,False,False,False
4,2.2 Final Adoption of Ordinances.20-056 Final ...,2,2,1,False,False,True
...,...,...,...,...,...,...,...
1571,4.1 20-1638 Update on Fine Amounts for Illegal...,1,4,45,False,True,True
1572,7. NEIGHBORHOOD SERVICES 7.1 20-1640 Police ...,0,5,45,False,True,False
1573,8. COMMUNITY & ECONOMIC DEVELOPMENT 8.1 20-1...,0,6,45,True,True,False
1574,9. REDEVELOPMENT – SUCCESSOR AGENCY \n The S...,0,7,45,False,True,False


### 3.5 save and read
+ to and from feather file
+ need pyarrow installed.

In [34]:
import time

start = time.time()
df_minutes.to_feather('city_SanJose_Minutes.feather')
df_read = pd.read_feather('city_SanJose_Minutes.feather')
print(time.time()-start)

df_read

1.8723499774932861


,content,subID,mainID,artID,hasDollar,hasItem,hasSection
0,CITY OF JET'\n\nsan Jose\n\nCAPITAL OF SILICON...,0,0,1,False,True,False
1,1. CEREMONIAL ITEMS\nNone provided.\n\nCity o...,0,1,1,False,False,False
2,2. CONSENT CALENDAR\nUpon motion by Councilme...,0,2,1,False,False,False
3,2.1 Approval of City Council Minutes.\nNone pr...,1,2,1,False,False,False
4,2.2 Final Adoption of Ordinances.\n20-056 \n\n...,2,2,1,False,True,True
...,...,...,...,...,...,...,...
1337,4.1 \n \n \n \n \n \n \n \n \n \n \n\n20-163...,1,4,45,False,True,True
1338,7. NEIGHBORHOOD SERVICES 7.1 \n \n \n \n \...,0,5,45,False,False,False
1339,8. COMMUNITY & ECONOMIC DEVELOPMENT 8.1 \n ...,0,6,45,True,True,False
1340,9. REDEVELOPMENT â SUCCESSOR AGENCY \n Th...,0,7,45,False,True,False


+ to and from csv file

In [16]:
import time

In [17]:
start = time.time()
df_minutes.to_csv('city_SanJose_Minutes.csv')
df_read = pd.read_csv('city_SanJose_Minutes.csv')
print(time.time()-start)

df_read

0.5056302547454834


,Unnamed: 0,content,subID,mainID,artID,hasDollar,hasItem,hasSection
0,0,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,0,0,1,False,True,False
1,1,1. CEREMONIAL ITEMSNone provided.\n\nCity of S...,0,1,1,False,False,False
2,2,2. CONSENT CALENDAR\nUpon motion by Councilmem...,0,2,1,False,False,False
3,3,2.1 Approval of City Council Minutes.None prov...,1,2,1,False,False,False
4,4,2.2 Final Adoption of Ordinances.20-056 Final ...,2,2,1,False,False,True
...,...,...,...,...,...,...,...,...
1571,1571,4.1 20-1638 Update on Fine Amounts for Illegal...,1,4,45,False,True,True
1572,1572,7. NEIGHBORHOOD SERVICES 7.1 20-1640 Police ...,0,5,45,False,True,False
1573,1573,8. COMMUNITY & ECONOMIC DEVELOPMENT 8.1 20-1...,0,6,45,True,True,False
1574,1574,9. REDEVELOPMENT – SUCCESSOR AGENCY \n The S...,0,7,45,False,True,False
